###🎯 What You'll Learn in This Video

✅ Access and visualize monthly temperature trends in Egypt (2000–2024) using ERA5-Land data

✅ Convert temperature from Kelvin to Celsius using Xarray

✅ Create a Plotly interactive line chart of Egypt’s monthly average temperature

✅ Resample the data to annual means using .resample()

✅ Generate yearly temperature maps for Egypt

In [ ]:
# Install xee library
! pip install xee

In [ ]:
# Import libraries
import ee
import geemap
import xarray as xr
import xee
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd

In [ ]:
# Authenticate and initialize Earth Engine
ee.Authenticate()

ee.Initialize(project = 'ee-nouribrahim25' , opt_url='https://earthengine-highvolume.googleapis.com')

In [ ]:
# Define Egypt Region
egy = ee.FeatureCollection ('projects/ee-nouribrahim25/assets/EGY-ADM0').geometry()
egy

In [ ]:
# Load ERA5-Land Monthly temperature data (Kelvin)
temp = (ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR')
      .filterBounds(egy)
      .filterDate('2000-01-01' , '2024-12-31')
      .select('temperature_2m')
      )

temp

In [ ]:
# Load into xarray using xee
ds = xr.open_dataset(
    temp,
    engine="ee",
    geometry= egy,
    crs="EPSG:4326",
    scale= 0.11,
)
ds

In [ ]:
# Convert Kelvin to Celsius
ds['temperature_2m'] = ds['temperature_2m'] - 273.15
ds

Monthly Time Series Plot (Plotly)

In [ ]:
#Aggregate to monthly mean over Egypt
monthly_temp = ds['temperature_2m'].mean(dim=["lon", "lat"])
monthly_temp

In [ ]:
#Convert to DataFrame
temp_df = monthly_temp.to_dataframe().reset_index()
temp_df.head()

In [ ]:
 #Plot with Plotly
fig = px.line(
    temp_df, x="time", y="temperature_2m",
    title="Monthly Mean Temperature in Egypt (2000–2024)",
    labels={"temperature_2m": "Temperature (°C)", "time": "Date"},
    template="plotly_white"
)
fig.show()

In [ ]:
# Add 12-month rolling average
temp_df['rolling'] = temp_df['temperature_2m'].rolling(12).mean()

In [ ]:
# Add rolling mean trace and update layout
fig.add_scatter(
    x=temp_df['time'], y=temp_df['rolling'],
    mode='lines', name='12-Month Rolling Avg',
    line=dict(color='red')
)

fig.update_xaxes(
    dtick="M12",
    tickformat="%Y",
    tickangle=0
)
fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.3,
        xanchor="center",
        x=0.5
    )
)
fig.show()

Annual Resampling & Temperature Maps

In [ ]:
# Resample the data into one year interval
annual = ds.resample(time = '1Y').mean('time')
annual

In [ ]:
# plot the annual data on the map
annual['temperature_2m'].plot(
    figsize=(18, 14),
    x = 'lon' ,
    y = 'lat' ,
    col = 'time' ,
    col_wrap=5 ,
    robust = True ,
    cmap = 'jet',
    vmin = 18 ,
    vmax = 30 ,
);